In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import datetime
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Data Cleaning

In [83]:
def kick_nan(x):
    x = str(x).replace(" ", "").lower()
    if (x == "nan") or (x == "免引水"):
        y = 0
    else:
        y = x
    return y

def get_dummy_column_wanted(df, all_col=[]):
    if len(all_col) > 0:
        for i in all_col:
            df = pd.concat([df, pd.get_dummies(df[i], prefix=[i])], axis=1)
        return df
    else:
        return df

def make_col_be_str(df, str_col=[]):
    if len(str_col) > 0:
        for i in str_col:
            df[i] = df[i].apply(lambda x:str(x))
        return df
    else:
        return df
        
def make_col_be_int(df, int_col=[]):
    if len(int_col) > 0:
        for i in int_col:
            df[i] = df[i].apply(lambda x:int(float(str(x))))
        return df
    else:
        return df

def make_col_be_float(df, float_col=[]):
    if len(float_col) > 0:
        for i in float_col:
            df[i] = df[i].apply(lambda x:float(str(x)))
        return df
    else:
        return df
    
def data_cleaning():
    df = pd.read_excel("khh1005.xlsx")
    
    # drop useless columns
    df["work_time"] = df["mean_work_time"]
    df = df.drop(["min_work_time", "max_work_time", "mean_work_time", "pilot_ready_time", "min_end_time", "max_end_time", "pilot_ready_time", "place1", "pilot2", "tug2_no", "tug3_no", "pier_info", "seven", "day"], axis=1)
    df = df[df["work_time"]>0]
    
    # fix the err data (kick out nan)
    for col in df.columns:
        df[col] = df[col].apply(lambda x:str(kick_nan(x)))
    
    # add features of the month, hour, year
    df["month"] = df["start_time"].apply(lambda x: int(str(x).replace("-", "").replace(":", "")[4:6]))
    df["day"] = df["start_time"].apply(lambda x: int(str(x).replace("-", "").replace(":", "")[6:8]))
    df["hour"] = df["start_time"].apply(lambda x: int(str(x).replace("-", "").replace(":", "")[8:10]))
    df["weekday"] = df["start_time"].apply(lambda x: int(datetime.datetime.strptime(str(x), "%Y-%m-%d%H:%M:%S").weekday()))
    
    # the datatype should be float, string, and integer
    df = make_col_be_float(df, ['front_weight','back_weight','weight_level','dist','wind'])
    df = make_col_be_str(df, ['sailing_status','park','reverse']) #,'start_time'
    df = make_col_be_int(df, ['pilot_wait_time', 'work_time', 'ship_no','port','place2','pilot1','tug1_no','tug_cnt','total_weight','avg_hp']) #, "month", "day", "hour"
    
    # drop useless columns
    df = df.drop(["start_time"], axis=1)
    
    # drop row with missing data
    df = df[(df.port != 0) & (df.port != 3)]
    df = df[(df.park != "0")]
    return df

In [84]:
df = data_cleaning()
print("shape of the df:", df.shape)
print("-"*80)
print("column names:", df.columns)
print("-"*80)
df.head()

shape of the df: (31178, 22)
--------------------------------------------------------------------------------
column names: Index(['ship_no', 'sailing_status', 'pilot_wait_time', 'port', 'place2',
       'pilot1', 'tug1_no', 'tug_cnt', 'total_weight', 'front_weight',
       'back_weight', 'weight_level', 'dist', 'wind', 'park', 'reverse',
       'avg_hp', 'work_time', 'month', 'day', 'hour', 'weekday'],
      dtype='object')
--------------------------------------------------------------------------------


,ship_no,sailing_status,pilot_wait_time,port,place2,pilot1,tug1_no,tug_cnt,total_weight,front_weight,...,dist,wind,park,reverse,avg_hp,work_time,month,day,hour,weekday
0,50091,o,0,1,1066,100,109,1,6094,6.0,...,4968.093286,0.15,o,0,3200,4,1,1,0,6
1,308684,i,4,1,1058,82,321,1,16232,6.0,...,3879.059264,1.00,o,0,3200,52,1,1,1,6
2,46698,i,8,2,1079,74,171,1,27968,10.0,...,1502.675679,1.00,l,0,5200,27,1,1,1,6
3,326585,i,5,2,1102,102,163,2,8562,4.0,...,2785.267130,0.75,o,0,3700,34,1,1,2,6
4,57126,i,5,2,1109,98,172,2,108069,11.5,...,1033.162198,0.75,r,0,5200,26,1,1,2,6


In [54]:
def pre_processing_before_train(df, dm_col, dp_col, pr_col, target_y_c, target_y_r):
    y_c = df[target_y_c]
    y_r = df[target_y_r]
    if dm_col != []:
        df = get_dummy_column_wanted(df, dm_col)
    else:
        df = df

    if dp_col+dm_col != []:
        x = df.drop(dp_col+dm_col, axis=1)
    else:
        x = df.drop([target_y_c, target_y_r], axis=1)
    print(x.columns)
    X_train, X_test, y_train_c, y_test_c = train_test_split(x, y_c, test_size=0.2, random_state=101)
    y_train_r, y_test_r = X_train[target_y_r], X_test[target_y_r]
    X_train, X_test = X_train.drop(target_y_r, axis=1), X_test.drop(target_y_r, axis=1)
 
    return X_train, X_test, y_train_c, y_test_c, y_train_r, y_test_r

# processing for status_sailing equal to "i"

In [89]:
df_1 = df[(df["sailing_status"] == "i")]
print("the shape of df_1", df_1.shape)
print("="*100)

# remove outlier
df_1_bl_p2std = int(np.mean(df_1.work_time) + np.std(df_1.work_time)*2)+1
df_1_bl_n2std = int(np.mean(df_1.work_time) - np.std(df_1.work_time)*2)+1
df_1 = df_1[(df_1.work_time<=df_1_bl_p2std) & (df_1_bl_n2std<df_1.work_time)]

print(df_1[["work_time"]].describe())
print("="*100)

the shape of df_1 (14981, 22)
          work_time
count  14759.000000
mean      35.605529
std       10.451598
min        9.000000
25%       27.000000
50%       35.000000
75%       43.000000
max       65.000000


In [90]:
df_1_med = df_1["work_time"].median()
dm_col_1c = ["port", "tug_cnt", "park", "reverse", "month", "hour", "weekday"]
dp_col_1c = ["front_weight", "back_weight", "tug1_no", "pilot1", "ship_no", "sailing_status", "place2", "day", "work_time_med", "pilot_wait_time"]
pr_col_1c = []
df_1["work_time_med"] = df_1["work_time"].apply(lambda x: 0 if x<=df_1_med else 1)
X1_train, X1_test, y1_train_c, y1_test_c, y1_train_r, y1_test_r = \
    pre_processing_before_train(df_1, dm_col_1c, dp_col_1c, pr_col_1c, "work_time_med", "work_time")

Index(['total_weight', 'weight_level', 'dist', 'wind', 'avg_hp', 'work_time',
       '['port']_1', '['port']_2', '['tug_cnt']_1', '['tug_cnt']_2',
       '['tug_cnt']_3', '['park']_l', '['park']_o', '['park']_r',
       '['reverse']_0', '['reverse']_1', '['month']_1', '['month']_2',
       '['month']_3', '['month']_4', '['month']_5', '['month']_6',
       '['month']_7', '['month']_8', '['month']_9', '['month']_10',
       '['month']_11', '['month']_12', '['hour']_0', '['hour']_1',
       '['hour']_2', '['hour']_3', '['hour']_4', '['hour']_5', '['hour']_6',
       '['hour']_7', '['hour']_8', '['hour']_9', '['hour']_10', '['hour']_11',
       '['hour']_12', '['hour']_13', '['hour']_14', '['hour']_15',
       '['hour']_16', '['hour']_17', '['hour']_18', '['hour']_19',
       '['hour']_20', '['hour']_21', '['hour']_22', '['hour']_23',
       '['weekday']_0', '['weekday']_1', '['weekday']_2', '['weekday']_3',
       '['weekday']_4', '['weekday']_5', '['weekday']_6'],
      dtype='object')


In [91]:
X1_train.shape

(11807, 58)

# processing for status_sailing equal to "t"

In [92]:
df_2 = df[(df["sailing_status"] == "t")]
print("the shape of df_2", df_2.shape)
print("="*100)

# remove outlier
df_2_bl_p2std = int(np.mean(df_2.work_time) + np.std(df_2.work_time)*2)+1
# df_2_bl_n2std = int(np.mean(df_2.work_time) - np.std(df_2.work_time)*2)+1
df_2 = df_2[(df_2.work_time<=df_2_bl_p2std)]

print(df_2[["work_time"]].describe())
print("="*100)

the shape of df_2 (1284, 22)
         work_time
count  1242.000000
mean     43.512077
std      13.898803
min       5.000000
25%      35.000000
50%      40.000000
75%      50.000000
max      91.000000


In [93]:
df_2_med = df_2["work_time"].median()
dm_col_2c = ["port", "tug_cnt", "park","reverse", "month", "hour", "weekday"]
dp_col_2c = ["front_weight", "back_weight", "ship_no", "pilot1", "sailing_status", "place2", "day", "work_time_med", "tug1_no", "pilot_wait_time"]
pr_col_2c = []
df_2["work_time_med"] = df_2["work_time"].apply(lambda x: 0 if x<=df_2_med else 1)
X2_train, X2_test, y2_train_c, y2_test_c, y2_train_r, y2_test_r = \
    pre_processing_before_train(df_2, dm_col_2c, dp_col_2c, pr_col_2c, "work_time_med", "work_time")

Index(['total_weight', 'weight_level', 'dist', 'wind', 'avg_hp', 'work_time',
       '['port']_1', '['port']_2', '['tug_cnt']_1', '['tug_cnt']_2',
       '['tug_cnt']_3', '['park']_l', '['park']_o', '['park']_r',
       '['reverse']_0', '['reverse']_1', '['month']_1', '['month']_2',
       '['month']_3', '['month']_4', '['month']_5', '['month']_6',
       '['month']_7', '['month']_8', '['month']_9', '['month']_10',
       '['month']_11', '['month']_12', '['hour']_0', '['hour']_1',
       '['hour']_2', '['hour']_3', '['hour']_4', '['hour']_5', '['hour']_6',
       '['hour']_7', '['hour']_8', '['hour']_9', '['hour']_10', '['hour']_11',
       '['hour']_12', '['hour']_13', '['hour']_14', '['hour']_15',
       '['hour']_16', '['hour']_17', '['hour']_18', '['hour']_19',
       '['hour']_20', '['hour']_21', '['hour']_22', '['hour']_23',
       '['weekday']_0', '['weekday']_1', '['weekday']_2', '['weekday']_3',
       '['weekday']_4', '['weekday']_5', '['weekday']_6'],
      dtype='object')


In [94]:
X2_train.shape

(993, 58)

# processing for status_sailing equal to "o"

In [95]:
df_3 = df[(df["sailing_status"] == "o")]
print("the shape of df_3", df_3.shape)
print("="*100)

# remove outlier
df_3_bl_p2std = int(np.mean(df_3.work_time) + np.std(df_3.work_time)*2)+1
# df_3_bl_n2std = int(np.mean(df_3.work_time) - np.std(df_3.work_time)*2)+1
df_3 = df_3[(df_3.work_time<=df_3_bl_p2std)]

print(df_3[["work_time"]].describe())
print("="*100)

the shape of df_3 (14913, 22)
          work_time
count  14259.000000
mean      15.424434
std        8.305709
min        1.000000
25%       10.000000
50%       15.000000
75%       20.000000
max       38.000000


In [96]:
df_3_med = df_3["work_time"].median()
dm_col_3c = ["port", "tug_cnt", "park", "reverse", "month", "hour", "weekday"]
dp_col_3c = ["front_weight", "back_weight", "pilot1", "tug1_no", "ship_no", "sailing_status", "place2", "day", "work_time_med", "pilot_wait_time"]
#pr_col_3c = ["total_weight", "dist", "weight_level", "wind", "avg_hp"]
pr_col_3c = []
df_3["work_time_med"] = df_3["work_time"].apply(lambda x: 0 if x<=df_3_med else 1)
X3_train, X3_test, y3_train_c, y3_test_c, y3_train_r, y3_test_r = \
    pre_processing_before_train(df_3, dm_col_3c, dp_col_3c, pr_col_3c, "work_time_med", "work_time")

Index(['total_weight', 'weight_level', 'dist', 'wind', 'avg_hp', 'work_time',
       '['port']_1', '['port']_2', '['tug_cnt']_1', '['tug_cnt']_2',
       '['tug_cnt']_3', '['park']_l', '['park']_o', '['park']_r',
       '['reverse']_0', '['reverse']_1', '['month']_1', '['month']_2',
       '['month']_3', '['month']_4', '['month']_5', '['month']_6',
       '['month']_7', '['month']_8', '['month']_9', '['month']_10',
       '['month']_11', '['month']_12', '['hour']_0', '['hour']_1',
       '['hour']_2', '['hour']_3', '['hour']_4', '['hour']_5', '['hour']_6',
       '['hour']_7', '['hour']_8', '['hour']_9', '['hour']_10', '['hour']_11',
       '['hour']_12', '['hour']_13', '['hour']_14', '['hour']_15',
       '['hour']_16', '['hour']_17', '['hour']_18', '['hour']_19',
       '['hour']_20', '['hour']_21', '['hour']_22', '['hour']_23',
       '['weekday']_0', '['weekday']_1', '['weekday']_2', '['weekday']_3',
       '['weekday']_4', '['weekday']_5', '['weekday']_6'],
      dtype='object')


In [97]:
X3_train.shape

(11407, 58)

# Training: Classification (Random Forest)

In [102]:
from sklearn import preprocessing, metrics
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle

In [100]:
# Define regression model
clf1 = RandomForestClassifier(max_depth=40, n_estimators=200,random_state=0, n_jobs=20)
clf2 = RandomForestClassifier(max_depth=40, n_estimators=200,random_state=0, n_jobs=20)
clf3 = RandomForestClassifier(max_depth=40, n_estimators=200,random_state=0, n_jobs=20)

# Train regression model
clf1.fit(X1_train, y1_train_c)
clf2.fit(X2_train, y2_train_c)
clf3.fit(X3_train, y3_train_c)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=40, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=20,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [103]:
# check accuracy
accuracy1 = metrics.accuracy_score(y1_test_c, clf1.predict(X1_test))
print("accuracy for df_1 classification", accuracy1)
accuracy2 = metrics.accuracy_score(y2_test_c, clf2.predict(X2_test))
print("accuracy for df_2 classification", accuracy2)
accuracy3 = metrics.accuracy_score(y3_test_c, clf3.predict(X3_test))
print("accuracy for df_3 classification", accuracy3)

accuracy for df_1 classification 0.8275745257452575
accuracy for df_2 classification 0.7349397590361446
accuracy for df_3 classification 0.7784011220196353


In [104]:
# Save model
with open('model_new/clf_1.pickle', 'wb') as f1:
    pickle.dump(clf1,f1)
with open('model_new/clf_2.pickle', 'wb') as f2:
    pickle.dump(clf2,f2)
with open('model_new/clf_3.pickle', 'wb') as f3:
    pickle.dump(clf3,f3)

# Training: Regression (Random Forest)

In [105]:
# Define model
regr1_0 = RandomForestRegressor(max_depth=25, random_state=0,n_estimators=100, n_jobs=10)
regr2_0 = RandomForestRegressor(max_depth=30, random_state=0,n_estimators=200, n_jobs=10)
regr3_0 = RandomForestRegressor(max_depth=50, random_state=0,n_estimators=100, n_jobs=10)
regr1_1 = RandomForestRegressor(max_depth=25, random_state=0,n_estimators=100, n_jobs=10)
regr2_1 = RandomForestRegressor(max_depth=30, random_state=0,n_estimators=200, n_jobs=10)
regr3_1 = RandomForestRegressor(max_depth=50, random_state=0,n_estimators=100, n_jobs=10)

In [106]:
# predict training data by classification model to know each row is higher or lower than median
pred1_train = clf1.predict(X1_train)
pred2_train = clf2.predict(X2_train)
pred3_train = clf3.predict(X3_train)

In [107]:
# Train regression model
regr1_0.fit( X1_train[pred1_train==0] , y1_train_r[pred1_train==0])
regr1_1.fit( X1_train[pred1_train==1] , y1_train_r[pred1_train==1])
regr2_0.fit( X2_train[pred2_train==0] , y2_train_r[pred2_train==0])
regr2_1.fit( X2_train[pred2_train==1] , y2_train_r[pred2_train==1])
regr3_0.fit( X3_train[pred3_train==0] , y3_train_r[pred3_train==0])
regr3_1.fit( X3_train[pred3_train==1] , y3_train_r[pred3_train==1])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=50,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=10,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [108]:
# Save model
with open('model_new/reg_1_0.pickle', 'wb') as f:
    pickle.dump(regr1_0,f)
with open('model_new/reg_1_1.pickle', 'wb') as f:
    pickle.dump(regr1_1,f)
with open('model_new/reg_2_0.pickle', 'wb') as f:
    pickle.dump(regr2_0,f)
with open('model_new/reg_2_1.pickle', 'wb') as f:
    pickle.dump(regr2_1,f)
with open('model_new/reg_3_0.pickle', 'wb') as f:
    pickle.dump(regr3_0,f)
with open('model_new/reg_3_1.pickle', 'wb') as f:
    pickle.dump(regr3_1,f)

# MAE of three status

In [109]:
# predict testing data by classification model to know each row is higher or lower than median
pred1_c = clf1.predict(X1_test)
pred2_c = clf2.predict(X2_test)
pred3_c = clf3.predict(X3_test)

In [110]:
# MAE of three status (seperated to lower or higher than median for each status)
print("status I")
print("mae for df_1 <= median:", mean_absolute_error(y1_test_r[pred1_c==0], regr1_0.predict(X1_test[pred1_c==0])))
print("mae for df_1 > median:", mean_absolute_error(y1_test_r[pred1_c==1], regr1_1.predict(X1_test[pred1_c==1])))
print("status T")
print("mae for df_2 <= median:", mean_absolute_error(y2_test_r[pred2_c==0], regr2_0.predict(X2_test[pred2_c==0])))
print("mae for df_2 > median:", mean_absolute_error(y2_test_r[pred2_c==1], regr2_1.predict(X2_test[pred2_c==1])))
print("status O")
print("mae for df_3 <= median:", mean_absolute_error(y3_test_r[pred3_c==0], regr3_0.predict(X3_test[pred3_c==0])))
print("mae for df_3 > median:", mean_absolute_error(y3_test_r[pred3_c==1], regr3_1.predict(X3_test[pred3_c==1])))

status I
mae for df_1 <= median: 4.515340206343565
mae for df_1 > median: 5.744414705756139
status T
mae for df_2 <= median: 7.874815950920247
mae for df_2 > median: 10.62912343470483
status O
mae for df_3 <= median: 4.711056622851365
mae for df_3 > median: 5.705160183066361


In [111]:
# Calculate MAE of three status
pred_1_0 = regr1_0.predict(X1_test[pred1_c == 0])
pred_1_1 = regr1_1.predict(X1_test[pred1_c == 1])
pred_1 = np.concatenate((pred_1_0, pred_1_1))
test_1 = np.concatenate((y1_test_r[pred1_c==0], y1_test_r[pred1_c==1]))

pred_2_0 = regr2_0.predict(X2_test[pred2_c == 0])
pred_2_1 = regr2_1.predict(X2_test[pred2_c == 1])
pred_2 = np.concatenate((pred_2_0, pred_2_1))
test_2 = np.concatenate((y2_test_r[pred2_c==0], y2_test_r[pred2_c==1]))

pred_3_0 = regr3_0.predict(X3_test[pred3_c == 0])
pred_3_1 = regr3_1.predict(X3_test[pred3_c == 1])
pred_3 = np.concatenate((pred_3_0, pred_3_1))
test_3 = np.concatenate((y3_test_r[pred3_c==0], y3_test_r[pred3_c==1]))

print("MAE status I:", mean_absolute_error(pred_1, test_1))
print("MAE status T:", mean_absolute_error(pred_2, test_2))
print("MAE status O:", mean_absolute_error(pred_3, test_3))

MAE status I: 5.12363215879674
MAE status T: 8.826102873030585
MAE status O: 5.01570126227209
